In [13]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

from utils import load_dataset
import pandas as pd
from datetime import date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
df = load_dataset()

100%|██████████| 23/23 [00:10<00:00,  2.30it/s]


In [33]:
def clean_dataset(dataset):
    """
    Cleans the dataset
    TODO: enumerate all cleaning steps, check EDA for details on these changes
    """

    dataset['TotalReviews'] = dataset['RatingDistTotal'].str.replace("total:", "").astype(int)

    dataset = dataset.dropna(subset= 'Description')
    dataset = dataset.drop(dataset[dataset['PublishYear'] <= pd.Timestamp.min.year].index)
    dataset = dataset.drop(dataset[dataset['PublishYear'] >= date.today().year].index)

    dataset['Publisher'] = dataset['Publisher'].fillna("Unknown")
    dataset['Language'] = dataset['Language'].fillna('eng')

    language_mask = dataset['Language'].str.startswith('en-')
    dataset.loc[language_mask, 'Language'] = 'eng'

    dataset = dataset[dataset.Language == 'eng']

    dataset = dataset.drop(columns=['PublishMonth', 'Language', 'PublishDay', 'ISBN', 'RatingDist1', 'RatingDist2', 'RatingDist3', 'RatingDist4', 'RatingDist5', 'Count of text reviews', 'RatingDistTotal', 'CountsOfReview', 'Id'])

    return dataset

In [34]:
df_clean = clean_dataset(df)

In [36]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143017 entries, 0 to 34758
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Name          1143017 non-null  object 
 1   Authors       1143017 non-null  object 
 2   Rating        1143017 non-null  float64
 3   PublishYear   1143017 non-null  int64  
 4   Publisher     1143017 non-null  object 
 5   PagesNumber   1143017 non-null  int64  
 6   Description   1143017 non-null  object 
 7   filename      1143017 non-null  object 
 8   TotalReviews  1143017 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 87.2+ MB


In [29]:
df_clean.head(3)

,Name,Authors,Rating,PublishYear,Publisher,Language,PagesNumber,Description,filename,TotalReviews
0,Barbarossa,Christopher Ailsby,3.0,2007,New Line Books,eng,192,"On 22 June 1941, Adolf Hitler launched Operati...",../data/book1900k-2000k.csv,1
2,Images of Barbarossa,Christopher Ailsby,3.5,2001,Ian Allan Ltd,eng,256,"On 22 June 1941, Adolf Hitler launched Operati...",../data/book1900k-2000k.csv,8
3,Romania After 2000: Five New Romanian Plays,Daniel Charles Gerould,4.0,2007,Martin E. Segal Theatre Center Publications,eng,226,The first anthology of new Romanian Drama publ...,../data/book1900k-2000k.csv,6


In [37]:
df_clean.index.size

1143017

In [38]:
df_clean.to_csv('../data/clean_dataset.csv')